# Data With Danny SQL Challenge
## Case Study #1 - Danny's Diner
<br>

- The 8 Week SQL Challenge is brought to the [Data With Danny](https://www.datawithdanny.com).
- To know more information about this challenge, [Click Here](https://8weeksqlchallenge.com/about/).

### Note: To install the IPython sql library:

```!pip install ipython-sql```

In [1]:
%%capture
%load_ext sql
%sql sqlite:///1_Dannys_Diner.db

**1. What is the total amount each customer spent at the restaurant?**

In [2]:
%%sql

SELECT sales.customer_id AS Customer,
    SUM(menu.price) AS Spendings
FROM sales
    Join menu USING (product_id)
GROUP BY Customer;

 * sqlite:///1_Dannys_Diner.db
Done.


Customer,Spendings
A,76
B,74
C,36


**2. How many days has each customer visited the restaurant?**

In [3]:
%%sql

SELECT customer_id AS Customer,
    COUNT(DISTINCT order_date) AS Days
FROM sales
GROUP BY Customer;

 * sqlite:///1_Dannys_Diner.db
Done.


Customer,Days
A,4
B,6
C,2


**3. What was the first item from the menu purchased by each customer?**

In [4]:
%%sql

SELECT DISTINCT sales.customer_id AS Customer,
    menu.product_name AS First_Item
FROM sales
    Join menu USING (product_id)
WHERE order_date = (SELECT MIN(order_date) FROM sales);

 * sqlite:///1_Dannys_Diner.db
Done.


Customer,First_Item
A,sushi
A,curry
B,curry
C,ramen


**4. What is the most purchased item on the menu and how many times was it purchased by all customers?**

In [5]:
%%sql

SELECT menu.product_name AS Most_Purchased,
    count(sales.product_id) AS Times_Purchased
FROM sales
    JOIN menu USING (product_id)
GROUP BY Most_Purchased
ORDER BY Times_Purchased DESC
LIMIT 1;

 * sqlite:///1_Dannys_Diner.db
Done.


Most_Purchased,Times_Purchased
ramen,8


**5. Which item was the most popular for each customer?**

In [6]:
%%sql

WITH orders AS
    (SELECT DISTINCT sales.customer_id AS Customer,
        menu.product_name AS Product,
        COUNT(*) AS Count
    FROM sales
        JOIN menu USING(product_id)
    GROUP BY 1, 2)
    
SELECT T1.Customer, T1.Product
FROM orders AS T1
    JOIN (SELECT Customer, MAX(Count) AS Max_Count FROM orders GROUP BY Customer) AS T2
    ON T1.Customer = T2.Customer AND T1.Count = T2.Max_Count;

 * sqlite:///1_Dannys_Diner.db
Done.


Customer,Product
A,ramen
B,curry
B,ramen
B,sushi
C,ramen


**6. Which item was purchased first by the customer after they became a member? (including their join date)**

In [7]:
%%sql

WITH after_joining AS
    (SELECT sales.customer_id AS Customer,
        menu.product_name AS Product,
        sales.order_date AS Date
    FROM sales
        JOIN menu USING (product_id)
        JOIN members USING (customer_id)
    WHERE sales.order_date >= members.join_date)

SELECT T1.Customer, T1.Product
FROM after_joining AS T1
    JOIN (SELECT Customer, MIN(Date) AS Min_Date FROM after_joining GROUP BY Customer) AS T2
    ON T1.Customer = T2.Customer AND T1.Date = T2.Min_Date;

 * sqlite:///1_Dannys_Diner.db
Done.


Customer,Product
A,curry
B,sushi


**7. Which item was purchased just before the customer became a member? (including their join date)**

In [8]:
%%sql

WITH after_joining AS
    (SELECT sales.customer_id AS Customer,
        menu.product_name AS Product,
        sales.order_date AS Date
    FROM sales
        JOIN menu USING (product_id)
        JOIN members USING (customer_id)
    WHERE sales.order_date <= members.join_date)

SELECT T1.Customer, T1.Product
FROM after_joining AS T1
    JOIN (SELECT Customer, MAX(Date) AS Max_Date FROM after_joining GROUP BY Customer) AS T2
    ON T1.Customer = T2.Customer AND T1.Date = T2.Max_Date;

 * sqlite:///1_Dannys_Diner.db
Done.


Customer,Product
A,curry
B,sushi


**8. What is the total items and amount spent for each member before they became a member? (including their join date)**

In [9]:
%%sql

SELECT sales.customer_id AS Customer,
    COUNT(menu.product_name) AS Items,
    SUM(menu.price) AS Price
FROM sales
    JOIN menu USING (product_id)
    JOIN members USING (customer_id)
WHERE sales.order_date <= members.join_date
GROUP BY 1;

 * sqlite:///1_Dannys_Diner.db
Done.


Customer,Items,Price
A,3,40
B,3,40


**9. If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?**

In [10]:
%%sql

WITH points AS
    (SELECT sales.customer_id AS Customer,
        menu.product_name AS Product,
        CASE WHEN menu.product_name = "sushi" THEN SUM(menu.price) * 10 * 2 ELSE SUM(menu.price) * 10 END AS Units 
    FROM sales
        JOIN menu USING (product_id)
    GROUP BY 1, 2)

SELECT Customer, SUM(Units)
FROM points
GROUP BY 1;

 * sqlite:///1_Dannys_Diner.db
Done.


Customer,SUM(Units)
A,860
B,940
C,360


**10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?**

In [11]:
%%sql

WITH members_points AS
    (SELECT sales.customer_id AS Customer,
        sales.order_date AS Date,
        menu.product_name AS Product,
        CASE
            WHEN sales.order_date BETWEEN members.join_date AND DATE(members.join_date, '+6 days') THEN SUM(menu.price) * 20
            WHEN menu.product_name = "sushi" THEN SUM(menu.price) * 20
            ELSE SUM(menu.price) * 10 END AS Units 
    FROM sales
        JOIN menu USING (product_id)
        JOIN members USING (customer_id)
    WHERE sales.order_date <= '2021-01-31'
    GROUP BY 1, 2, 3)

SELECT Customer, SUM(Units)
FROM members_points
GROUP BY 1;

 * sqlite:///1_Dannys_Diner.db
Done.


Customer,SUM(Units)
A,1370
B,820


## Bonus Questions:

### __Join All The Things To recreate [this table](https://8weeksqlchallenge.com/case-study-1/#:~:text=Bonus%20Questions-,Join%20All%20The%20Things,-The%20following%20questions).__

In [12]:
%%sql

SELECT sales.customer_id, sales.order_date, menu.product_name, menu.price,
    CASE WHEN sales.customer_id IN (SELECT customer_id FROM members) AND sales.order_date >= members.join_date THEN 'Y'
    ELSE 'N' END AS member
FROM sales
    JOIN menu USING (product_id)
    LEFT JOIN members USING (customer_id);

 * sqlite:///1_Dannys_Diner.db
Done.


customer_id,order_date,product_name,price,member
A,2021-01-01,sushi,10,N
A,2021-01-01,curry,15,N
A,2021-01-07,curry,15,Y
A,2021-01-10,ramen,12,Y
A,2021-01-11,ramen,12,Y
A,2021-01-11,ramen,12,Y
B,2021-01-01,curry,15,N
B,2021-01-02,curry,15,N
B,2021-01-04,sushi,10,N
B,2021-01-11,sushi,10,Y


### __Rank All The Things To recreate [this table](https://8weeksqlchallenge.com/case-study-1/#:~:text=N-,Rank%20All%20The%20Things,-Danny%20also%20requires).__

In [13]:
%%sql

WITH cte AS
    (SELECT sales.customer_id, sales.order_date, menu.product_name, menu.price,
        CASE WHEN sales.customer_id IN (SELECT customer_id FROM members) AND sales.order_date >= members.join_date THEN 'Y'
        ELSE 'N' END AS member
    FROM sales
        JOIN menu USING (product_id)
        LEFT JOIN members USING (customer_id))

SELECT *, CASE WHEN member = 'N' THEN null
    ELSE  RANK() OVER(PARTITION BY customer_id, member ORDER BY order_date)
    END AS ranking
FROM cte;

 * sqlite:///1_Dannys_Diner.db
Done.


customer_id,order_date,product_name,price,member,ranking
A,2021-01-01,sushi,10,N,None
A,2021-01-01,curry,15,N,None
A,2021-01-07,curry,15,Y,1
A,2021-01-10,ramen,12,Y,2
A,2021-01-11,ramen,12,Y,3
A,2021-01-11,ramen,12,Y,3
B,2021-01-01,curry,15,N,None
B,2021-01-02,curry,15,N,None
B,2021-01-04,sushi,10,N,None
B,2021-01-11,sushi,10,Y,1
